In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.onnx #torch신경망을 onnx로 변환
import onnx
import onnxruntime

In [2]:
# torch 신경망 불러오는 목적

class NN(nn.Module) :
    def __init__(self, vector_tensor) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(vector_tensor, freeze = True, padding_idx = 0) #freeze : 해당 벡터사전 고정, padding_idx = 패딩 인스스
        self.rnn = nn.LSTM(200,20,batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(20, 2)
        )
    def forward(self, x) :
        x = self.embedding(x)
        y, h = self.rnn(x)
        y = y[:,-1,:]
        y = self.f(y)
        return y

In [3]:
torch_F = torch.load("korean_movie_2.pt", weights_only=False)
print(torch_F)

NN(
  (embedding): Embedding(30187, 200, padding_idx=0)
  (rnn): LSTM(200, 20, batch_first=True)
  (f): Sequential(
    (0): Linear(in_features=20, out_features=2, bias=True)
  )
)


In [4]:
#변환하기 전 더미 입력값
x = torch.randint(0,30187,size = (1,30)).type(torch.long)
print(x)
print(x.shape)

tensor([[ 8584,   623,   262, 19086, 13338,  5349, 13080,  5136, 21274, 25445,
         26824, 16946, 14903, 12644, 28889, 20574, 11081, 30033, 11209, 14279,
          1428,  9359,   559, 29866,  9391, 27034, 22940, 17678,  9278,  7080]])
torch.Size([1, 30])


In [5]:
torch.onnx.export(torch_F, #신경망
                 x, #입력값 tensor 형태
                 "onnx_korean_nn.onnx", #저장할 파일 이름
                 export_params = True, #모델 가중치(w, b)값을 저장할지
                 opset_version = 10, #ONNX 버전
                 do_constant_folding = True,
                 input_names = ['input'], #입력 텐서의 이름
                 output_names = ['output'], #출력 텐서의 이름
                 dynamic_axes = {'input' : {0 : 'b'}, 'output' : {0 : 'b'}}
                 )

C:\Users\Administrator\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [6]:
onnx_model = onnx.load("onnx_korean_nn.onnx")
onnx.checker.check_model(onnx_model)

#메세지 안뜨면 onnx 변환이 잘 된것

In [7]:
onnx_F = onnxruntime.InferenceSession("onnx_korean_nn.onnx", provider = ["CPUExcutionProvider"])

numpy_x = np.random.randint(0,30187,size = (1,30)).astype(np.int64)
torch_x = torch.tensor(numpy_x, dtype = torch.long)

ort_inputs = {onnx_F.get_inputs()[0].name : numpy_x} #onnx에서 x 입력방법, dict 형태로 입력해야 함
onnx_y = onnx_F.run(None, ort_inputs)

torch_y = torch_F(torch_x)

print("torch 함수의 결과값 : ", torch_y.detach().numpy())
print("onnx 함수의 결과값 : ", onnx_y[0])

torch 함수의 결과값 :  [[ 1.6751504 -1.9235777]]
onnx 함수의 결과값 :  [[ 1.6751499 -1.9235772]]
